## Python Importation

In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import bk.compute
import bk.plot
import bk.load

import scipy.signal
import neuroseries as nts

## Magic Command

In [2]:
%matplotlib qt

## Session Init

In [5]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

In [10]:
bk.load.current_session()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

## Loading data

In [6]:
neurons, metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


## Computing spectrograms

In [5]:
start = 15516.0
end = 16405.0

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

In [6]:
lfp = bk.load.lfp(start,end,166,13)
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=10000,noverlap=5000)
t = t +lfp.as_units('s').index[0]
spec = Sxx[f<20]
spec_corr = np.corrcoef(spec,rowvar=False)

Load LFP from Rat08-20130713.lfp


## Computing Neurons correlations

In [33]:
# start = 15516.0
# end = 16405.0

start = 2300
end = 2500

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

# stru = ['Hpc','BLA','Pir']
# channel = [13,101,80]

stru =  ['BLA']
channel = [103]
for n,(s,c) in enumerate(zip(stru,channel),0):
    lfp = bk.load.lfp(start,end,166,c)
    f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=1000,noverlap=250)
    t = t +lfp.as_units('s').index[0]
    spec = Sxx[f<100]
    spec_corr = np.corrcoef(spec,rowvar=False)
    
    stru_neurons,t_n = bk.compute.binSpikes(neurons[metadata.Region == s],2,start,end)
    neurons_corr = np.corrcoef(stru_neurons,rowvar=False)
    
    plt.figure()
    sub = plt.subplot(4,1,1)
    plt.title('LFP')
    plt.plot(lfp.as_units('s'))
    bk.plot.intervals(states['sws'],col='grey')
    bk.plot.intervals(states['Rem'])
   
    plt.subplot(4,1,2,sharex=sub)
    plt.title('Spectrogram')
    plt.pcolormesh(t, f, Sxx)
    plt.ylim(0,100)
    plt.clim(0,5000)
    
    
    plt.subplot(4,1,3,sharex=sub)
    plt.title('SpectrogramCorr')
    plt.pcolormesh(t,t,spec_corr)
    plt.clim(0,1)
    
    plt.subplot(4,1,4,sharex = sub)
    plt.title('SpikeTrainCorr') 
    plt.pcolormesh(t_n,t_n,neurons_corr)
    plt.xlim(start,end)
    
    plt.tight_layout()
    plt.suptitle(s)

Load LFP from Rat08-20130713.lfp


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.


In [32]:
stru_neurons.shape

(94, 999)

In [15]:
fig, axes = plt.subplots(nrows=3, ncols=2)

In [16]:
for ax in axes.flatten():
    ax.plot(lfp)

In [48]:
end  = nts.Ts(np.array(states['sws'].end+5_000_000))
a = states['Rem'].in_interval(end)

In [49]:
a

array([nan,  0.,  1.,  2.,  3.,  4.,  5.,  6., nan,  9., 10., nan, 11.,
       12., 13., 14., 15., 16., 17., 18., 19., nan, nan, 20., 21., 22.,
       23., 24., 25., 26., nan, 27., 28., 29.])

pandas.core.indexes.range.RangeIndex

In [ ]:
def state_transitions(states):
    '''
        states : dict of nts.Interval_Set
        
        This function compute transition in between Intervals in a dict.
        It returns a new dict with intervals and when the transition occurs
    '''
    
    